In [7]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show, output_file
import matplotlib.cm as cm
import matplotlib as mpl
from bokeh.plotting import hplot
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, PreviewSaveTool
)

In [8]:
import pickle
Data_dir='../../../Data/Weather'
stations=pickle.load(open(Data_dir+'/stations.pkl','r'))
stations.head()

,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390


In [9]:
#this code is using pandas dataframes
lons=stations.ix[:,'longitude'].values
lats=stations.ix[:,'latitude'].values
elev=stations.ix[:,'elevation'].values
station_names=stations.index.values
ll=len(lons)
lonmin=-180;lonmax=180;latsmin=-80;latsmax=80;
select=(lons>lonmin) * (lons<lonmax)*(lats>latsmin)*(lats<latsmax)
print sum(select)
station_names=station_names[select]
lons=lons[select]
lats=lats[select]
elev=elev[select]
elev=[min(3000,max(e,0))/3000. for e in elev]
print len(lons),len(lats),len(station_names),len(elev)

85273
85273 85273 85273 85273


In [10]:
arr = np.array(zip(lons, lats, elev))
indices = np.random.choice(xrange(len(arr)), 4000, replace=False)
new_arr = arr[indices]
lons = new_arr[:,0]
lats = new_arr[:,1]
elev = new_arr[:,2]

### Define fields to display in map

In [11]:
variable1 = elev
variable2 = elev

In [12]:
colors = [
    "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*cm.hot(mpl.colors.Normalize()(variable1))
]
colors2 = [
    "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*cm.viridis(mpl.colors.Normalize()(variable2))
]

### Define output HTML

In [15]:
output_file("plot.html", title="USA Elevation plot", mode="cdn")

In [16]:
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, PreviewSaveTool
)

map_options = GMapOptions(lat=37.09024, lng=-95.712891, map_type="terrain", zoom=3)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options)

plot2 = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options)

source = ColumnDataSource(
    data=dict(
        lat=lats,
        lon=lons,
        col=colors
    )
)

source2 = ColumnDataSource(
    data=dict(
        lat=lats,
        lon=lons,
        col=colors2
    )
)

circle = Circle(x="lon", y="lat", size=5, fill_color="col", fill_alpha=1, line_color=None,)
plot.add_glyph(source, circle) # Plot for variable 1
plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), PreviewSaveTool())

plot2.add_glyph(source2, circle) # Plot for variable 2
plot2.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), PreviewSaveTool())

p = hplot(plot,plot2)
show(p)
